# Tasks

Now, put neural networks into action. You are tasked with building a neural network using data from this [kaggle competition](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data?select=sample_submission.csv). To complete the assignment, train and evaluate your model using only the train.csv.

 ### Required Task:
Be sure to perform a train test split on the train.csv so you can evaluate your models.
Create and evaluate 3 iterations of a deep learning model to predict housing prices using the techniques you have learned to optimize your model's performance. Be sure to include some form of regularization with at least one model.
Select your best model!

### Optional Task
Use your best model to make predictions using the features in test.csv.
Submit to the Kaggle competition to see how your did!
Include a screenshot of your results from the Kaggle competition inserted in a markdown cell at the bottom of your notebook.

# Data Imports

In [ ]:
filename = ''